**Step 1: Find all patients with condition X from three sources**

**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Step 1.1: Find 

In [2]:
#connect to UMLS
import mysql.connector
import pprint

try:
  cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  print
  #cnx.close()

In [3]:
#Define functions
def findMeshAndICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='MSH' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        msh = "Undef"
        for code in cursor:
            msh = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10CM' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        for code in cursor:
            icd10 = code
        return (msh[0],icd10[0])
    return ("Undef","Undef") 


In [4]:
findMeshAndICD10('Asthma')

(u'D001249', u'J45.90')

In [ ]:
## FHIR patient data
import requests
base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
patients = base_url+"/Patient"+"?_count=50"

req = requests.get(patients)

In [6]:
## Pull patients matching a list of ICD codes
from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator
query = GreenTranslator ().get_query ()

patients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
pprint.pprint (patients)

NameError: name 'pprint' is not defined

In [ ]:
meds = {}
for x in patients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)